
# Algoritmo de triangulação

A triangulação do polígono é o primeiro ingrediente da nossa solução para o problema da galeria de arte. O objetivo desse notebook é explicar como fazer isso através do algoritmo de *ear decomposition* (decomposição por orelhas).  
Primeiro, lemos o polígono de um arquivo de entrada

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random as rn
rn.seed()
# Selecione o polígono entre in0, in1, in2, ... in6
polygon = "in6"

# Paleta de cores + transparência
custom_colors = [
    'rgba(12, 62, 94, 0.6)', 'rgba(21, 91, 135, 0.6)', 'rgba(45, 146, 209, 0.6)',
    'rgba(116, 187, 232, 0.6)', 'rgba(151, 209, 244, 0.6)',
    'rgba(12, 94, 80, 0.6)', 'rgba(21, 135, 116, 0.6)', 'rgba(46, 209, 181, 0.6)',
    'rgba(116, 232, 212, 0.6)', 'rgba(151, 244, 229, 0.6)'
]

# Leitura da entrada

with open("pols/" + polygon) as f:
    L = list(f.readline().split())

N = int(L[0])
L = L[1:]

points = []

for i in range(0, N):
    s, t = L[2*i], L[2*i+1]
    a, b = map(int, s.split('/'))
    c, d = map(int, t.split('/'))
    points.append((a/b, c/d))


# Plotar o polígono

def plt_pol():
    plt.figure(figsize=(5, 5))
    plt.axis('equal')
    xs = [x for (x, y) in points]
    ys = [y for (x, y) in points]
    plt.fill(xs, ys)
    plt.show()

plt_pol()

Precisamos de algumas primitivas geométricas

In [ ]:
# Primitiva de sentido horário/anti-horário

def area(i, j, k):
    (ix, iy) = points[i]
    (jx, jy) = points[j]
    (kx, ky) = points[k]
    
    (vx, vy) = (jx - ix, jy - iy)
    (wx, wy) = (kx - jx, ky - jy)
    
    return vx*wy - vy*wx   

def ccw(i, j, k, eps=1e-9):
    return area(i, j, k) >= eps


# Primitiva de estar dentro de um triângulo (l dentro de i, j, k)

def inside(i, j, k, l, eps=1e9):
    return (area(i, j, l) >= 0) and (area(j, k, l) >= 0) and (area(k, i, l) >= 0)

Segue o algoritmo que decide se um ponto é uma orelha

In [ ]:
# Sou ponta de orelha? Complexidade O(N)

def eartest(j):
    i, k = prev[j], prox[j]
    
    if not ccw(i, j, k):
        return False
    
    for l in range(0, N):
        if(l==i or l==j or l==k):
            continue
        if inside(i, j, k, l):
            return False

    return True

Segue o algoritmo de triangulação

In [ ]:
# Triangulação. Complexidade: O(N^2)

def triangulate():  
    
    # Adjacência no polígono
    
    global prev, prox
    prev, prox = [], []
    for i in range(0, N):
        prev.append((i-1)%N)
        prox.append((i+1)%N)
    
    # Inicialmente, quem é orelha?
    
    ear = []
    
    for j in range(0, N):
        ear.append(eartest(j))
    
    triangulation = []
    remaining = N
    
    # Loop principal
    
    while remaining >= 3:
    
        # Encontrar uma orelha
        i = -1
        for j in range(0, N):
            if ear[j]:
                i = j
                break
        
        # Adicionar o triangulo encontrado
        triangulation.append((prev[i], i, prox[i]))
        ear[i] = False
        
        # Remover i do polígono
        prox[prev[i]] = prox[i]
        prev[prox[i]] = prev[i]
        remaining -= 1
        
        # Atualizar "status de orelha" dos vizinhos
        ear[prev[i]] = eartest(prev[i])
        ear[prox[i]] = eartest(prox[i])
        
    return triangulation

Agora, vamos plotar o resultado

In [ ]:
def plt_tring():
    plt.figure(figsize=(10, 10))
    plt.axis('equal')
    for t in triangulate():
        l = [points[i] for i in t]
        txs = [x for (x, y) in l]
        tys = [y for (x, y) in l]
        plt.fill(txs, tys, edgecolor='black', linewidth=0.1)
    plt.show()

def tring_anim(i):
    if(i==0):
        return arts
    t = tt[i-1]
    for p in range(1):
        l = [points[i] for i in t]
        txs = [x for (x, y) in l]
        tys = [y for (x, y) in l]
        pol = plt.fill(txs, tys, edgecolor='black', linewidth=0.1)[0]
        arts.append(pol)
    return arts

def anim():
    global tt, arts
    tt = triangulate()
    fig = plt.figure(figsize=(8, 8))
    xs, ys = [x for (x, y) in points], [y for (x, y) in points]
    ax = fig.add_subplot(autoscale_on=False, xlim=(min(xs), max(xs)), ylim=(min(ys), max(ys)))
    ax.set_aspect('equal')
    arts = [plt.fill(xs, ys, color='black')[0]]
    global ani
    ani = animation.FuncAnimation(fig, tring_anim, N-1, interval=1e4/N, blit=True, repeat=False)
    plt.show()

anim()

# Problema da Galeria de Arte

Para computar qual a menor quantidade de câmeras necessárias para vigiar toda a galeria de arte (representada pelo polígono) precisamos tentar cobrir a maior quantidade de triângulos com a menor quantidade de câmeras.

## Grafo Dual
Esse problema pode ser resolvido de forma mais simples se fizermos o grafo dual do nosso polígono triangulado. Vamos adotar cada triângulo como um vértice e dois vértices são adjascentes se seus triângulos compartilham uma lateral.

Segue o algoritmo que computa o grafo dual da triangulação

In [ ]:
import networkx as nx # lib de grafos

triangulation = triangulate()

# Primeiro precisamos identificar quais triângulos possuem laterais em comum
def share_edge(tri1, tri2):
    # Cada triângulo possui 3 vértices, se dois triângulos compartilham
    # 2 vértices então eles possuem uma lateral em comum
    
    shared = set(tri1) & set(tri2)
    return len(shared) == 2

def compute_dual_graph(triangulation):
    graph = nx.Graph()
    num_triangles = len(triangulation)
    
    # Adicionando os vértices do grafo
    for i in range(num_triangles):
        graph.add_node(i)
    
    # Adicionando arestas se os triângulos compartilham uma lateral
    for i in range(num_triangles):
        for j in range(i + 1, num_triangles):
            if share_edge(triangulation[i], triangulation[j]):
                graph.add_edge(i, j)
                
    
    return graph

Agora vamos plotar o resultado

In [ ]:
import plotly.graph_objects as go

def plot_dual_graph(triangulation, dual_graph):
    pos = {}
    for i, tri in enumerate(triangulation):
        x = sum(points[v][0] for v in tri) / 3
        y = sum(points[v][1] for v in tri) / 3
        pos[i] = (x, y)
    
    fig = go.Figure()

    # Garante que vamos ter cores o suficiente para colorir
    num_tri = len(triangulation)
    colors = (custom_colors * (num_tri // len(custom_colors) + 1))[:num_tri]

    # Triangulação
    tri_traces = []
    for i, t in enumerate(triangulation):
        t_points = [points[v] for v in t] + [points[t[0]]]
        xs, ys = zip(*t_points)
        color = colors[i]
        trace = go.Scatter(
            x=xs, y=ys, mode='lines', 
            hoverinfo='none',
            line=dict(color='#0c5e50', width=0.4),
            fill='toself',
            fillcolor=color,
            visible=True
        )
        fig.add_trace(trace)
        tri_traces.append(trace)

    # Dual
    edge_x = []
    edge_y = []
    for edge in dual_graph.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]
    
    dual_edges = go.Scatter(
        x=edge_x, y=edge_y, 
        line=dict(width=0.6, color='#0c5e50'), 
        hoverinfo='none', 
        mode='lines',
        visible=True
    )
    fig.add_trace(dual_edges)
    
    node_x = []
    node_y = []
    for node in dual_graph.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
    
    dual_nodes = go.Scatter(
        x=node_x, y=node_y, 
        hoverinfo='none',
        mode='markers',
        marker=dict(size=6, color='#0c5e50', line=dict(width=0.5, color='#0c5e50')),
        visible=True
    )
    fig.add_trace(dual_nodes)

    fig.update_layout(
        title="Grafo Dual da Triangulação", 
        showlegend=False,
        xaxis=dict(scaleanchor="y", scaleratio=1),
        yaxis=dict(scaleanchor="x", scaleratio=1),
        width=700,
        height=900,
        updatemenus=[
            {
                'buttons': [
                    {
                        'label': 'Ambos',
                        'method': 'update',
                        'args': [{'visible': [True] * len(tri_traces) + [True, True]}]
                    },
                    {
                        'label': 'Triangulação',
                        'method': 'update',
                        'args': [{'visible': [True] * len(tri_traces) + [False, False]}]
                    },
                    {
                        'label': 'Dual',
                        'method': 'update',
                        'args': [{'visible': [False] * len(tri_traces) + [True, True]}]
                    }
                ],
                'direction': 'down',
                'showactive': True
            }
        ]
    )

    fig.show()

dual_graph = compute_dual_graph(triangulation)
plot_dual_graph(triangulation, dual_graph)

## Coloração de vértices
Agora que temos o grafo dual da triangulação, precisamos decidir onde e quantas câmeras devemos colocar na galeria.
O grafo dual nos dá a informação do relacionamento dos triângulos e vamos usar isso para colorir gulosamente os vértices da triangulação da seguinte forma:

- Primeiro escolhemos um triângulo e colorimos cada um de seus vértices com uma cor
- Depois vamos visitar um de seus vizinhos (obtemos isso do grafo dual) e tentar colori-lo
- Porém, dois triângulos só podem compartilhar 2 vértices, logo existe apenas uma cor possível para o vértice não colorido dos vizinhos de cada triângulo

Seque o algoritmo de 3-coloração dos triângulos

In [ ]:
colors = {}  # dicionário -> triangulo: [a,b,c], onde a,b,c são as cores de seus vértices

color_map = {0: 'red', 1: 'blue', 2: 'green'}  # mapeamento das cores
reverse_color_map = {'red': 0, 'blue': 1, 'green': 2}  # mapeamento reverso

vertex_colors = {}  # coloração final da triangulação -> vertice: cor (string)

def dfs(current):
    triangle = triangulation[current]
    available = [0, 1, 2]
    
    # removendo cores já utilizadas nos vértices compartilhados
    for neighbor in dual_graph.neighbors(current):
        if neighbor in colors:
            n_colors = colors[neighbor]
            shared_vertices = set(triangle) & set(triangulation[neighbor])
            
            for vertex in shared_vertices:
                if vertex in vertex_colors:
                    color_code = reverse_color_map[vertex_colors[vertex]]
                    available.remove(color_code)
                       
    # Colorindo o triângulo atual com as cores disponíveis
    for color in available:
        for vertex in triangle:
            if vertex not in vertex_colors:
                vertex_colors[vertex] = color_map[color]
                break
    
    colors[current] = [0,1,2]
        
    # Chamada recursiva da dfs nos triângulos vizinhos que ainda não foram coloridos
    for neighbor in dual_graph.neighbors(current):
        if neighbor not in colors:
            dfs(neighbor)

# Chamada inicial da dfs na raiz do dual para colorir a triangulação
dfs(0)


Agora vamos plotar o resultado da coloração

In [ ]:
def plot_triangulation(triangulation, vertex_colors):
    plt.figure(figsize=(6, 8))
    for tri in triangulation:
        t_points = [points[v] for v in tri] + [points[tri[0]]]
        xs, ys = zip(*t_points)
        plt.fill(xs, ys, edgecolor='black', linewidth=0.5, facecolor='none')
    
    for vertex, color in vertex_colors.items():
        x, y = points[vertex]
        plt.plot(x, y, 'o', color=color, markersize=4)
    
    plt.title("Coloração da triangulação")
    plt.show()

plot_triangulation(triangulation, vertex_colors)

Agora para saber a menor quantidade de câmeras necessárias, precisamos olhar para a quantidade vértices da cor que coloriu menos vértices pois ela cobriu a maior quantidade de triângulos ao mesmo tempo, dado que todos os triângulos tem 1 vértice de cada cor.

In [ ]:
def plot_triangulation_min_color(triangulation, vertex_colors):
    plt.figure(figsize=(6, 8))
    
    # Contagem de vértices por cor
    color_counts = {'red': 0, 'blue': 0, 'green': 0}
    
    for color in vertex_colors.values():
        color_counts[color] += 1
    
    # Cor com a menor nro de vértices
    min_color = min(color_counts, key=color_counts.get)
    
    for tri in triangulation:
        t_points = [points[v] for v in tri] + [points[tri[0]]]
        xs, ys = zip(*t_points)
        plt.fill(xs, ys, edgecolor='black', linewidth=0.5, facecolor='none')
    
    for vertex, color in vertex_colors.items():
        if color == min_color:
            x, y = points[vertex]
            plt.plot(x, y, 'o', color=color, markersize=4)
    
    plt.title(f"Localização das câmeras (cor: {min_color})")
    plt.show()
    
plot_triangulation_min_color(triangulation, vertex_colors)